In [149]:
import re
import pronouncing

file = open("transcription.txt", "r")
corpus = file.read()
file.close()


original_text = [lines for lines in  corpus.split("\n") if len(lines.strip()) > 0]
splits_list = [re.sub(r'[^\w\s]', '', text).lower().split() for text in original_text]
splits_list = [[word for word in words if not(any(char.isdigit() for char in word))] for words in splits_list]
splits = [' '.join(l) for l in splits_list]

split_count = len(original_text)

print(split_count)
print(splits[2])
print(original_text[2])

6
because neither party is blameless for the decisions that led to this problem both parties have a responsibility to solve it and over the last several months thats what weve been trying to do i wont bore you with the details of every plan or proposal but basically the debate has centered around two different approaches
Because neither party is blameless for the decisions that led to this problem, both parties have a responsibility to solve it.  And over the last several months, that’s what we’ve been trying to do.  I won’t bore you with the details of every plan or proposal, but basically, the debate has centered around two different approaches.


In [150]:
import json

viseme_to_cmu = {
    'V2': ['ay', 'ah'],
    'V3': ['ey', 'eh', 'ae'],
    'V4': ['er'],
    'V5': ['iy', 'ih', 'y'],
    'V6': ['uw', 'uh', 'w'],
    'V7': ['ao', 'aa', 'oy', 'ow'],
    'V8': ['aw'],
    'V9': ['g', 'hh', 'k', 'ng'],
    'V10': ['r'],
    'V11': ['l', 'd', 'n', 't'],
    'V12': ['s', 'z'],
    'V13': ['ch', 'sh', 'jh', 'zh'],
    'V14': ['th', 'dh'],
    'V15': ['f', 'v'],
    'V16': ['m', 'b', 'p']
}
cmu_to_viseme = {}
all_phonemes = []

for key in viseme_to_cmu:
    for cmu in viseme_to_cmu[key]:
        cmu_to_viseme[cmu] = key
        all_phonemes.append(cmu)
cmu_to_viseme = {key: value for key, value in sorted(cmu_to_viseme.items())}

print(len(all_phonemes))
print(json.dumps(cmu_to_viseme, indent=4))

39
{
    "aa": "V7",
    "ae": "V3",
    "ah": "V2",
    "ao": "V7",
    "aw": "V8",
    "ay": "V2",
    "b": "V16",
    "ch": "V13",
    "d": "V11",
    "dh": "V14",
    "eh": "V3",
    "er": "V4",
    "ey": "V3",
    "f": "V15",
    "g": "V9",
    "hh": "V9",
    "ih": "V5",
    "iy": "V5",
    "jh": "V13",
    "k": "V9",
    "l": "V11",
    "m": "V16",
    "n": "V11",
    "ng": "V9",
    "ow": "V7",
    "oy": "V7",
    "p": "V16",
    "r": "V10",
    "s": "V12",
    "sh": "V13",
    "t": "V11",
    "th": "V14",
    "uh": "V6",
    "uw": "V6",
    "v": "V15",
    "w": "V6",
    "y": "V5",
    "z": "V12",
    "zh": "V13"
}


In [151]:
expl = ""

for key in viseme_to_cmu:
    if len(viseme_to_cmu[key]) == 1:
        continue
    expl += " ".join([f"/{k}/" for k in viseme_to_cmu[key]]) + " are likely to be confused with one another.\n"

print(expl)

/ay/ /ah/ are likely to be confused with one another.
/ey/ /eh/ /ae/ are likely to be confused with one another.
/iy/ /ih/ /y/ are likely to be confused with one another.
/uw/ /uh/ /w/ are likely to be confused with one another.
/ao/ /aa/ /oy/ /ow/ are likely to be confused with one another.
/g/ /hh/ /k/ /ng/ are likely to be confused with one another.
/l/ /d/ /n/ /t/ are likely to be confused with one another.
/s/ /z/ are likely to be confused with one another.
/ch/ /sh/ /jh/ /zh/ are likely to be confused with one another.
/th/ /dh/ are likely to be confused with one another.
/f/ /v/ are likely to be confused with one another.
/m/ /b/ /p/ are likely to be confused with one another.



In [152]:
import random
def random_phoneme(original_phoneme, prob_viseme, prob_different):
    if prob_viseme + prob_different > 1:
        raise Exception()
    actual_prob = random.random()
    if actual_prob < prob_viseme:
        visemes_excluding_curr = [item for item in viseme_to_cmu[cmu_to_viseme[original_phoneme]] if not(item == original_phoneme)]
        if len(visemes_excluding_curr) == 0:
            return original_phoneme
        return random.choice(visemes_excluding_curr)
    elif actual_prob < prob_viseme + prob_different:
        phonemes_excluding_curr = [item for item in all_phonemes if not(item == original_phoneme)]
        return random.choice(phonemes_excluding_curr)
    else:
        return original_phoneme

In [153]:
def string_to_phonemes(original_text, prob_viseme, prob_different):
    words = original_text.split(" ")

    word_pronounciations = []

    for word in words:
        pronouncing_word = pronouncing.phones_for_word(word)
        if len(pronouncing_word) > 0:
            pronounciation = pronouncing_word[0]
            pronounciation = ''.join([i for i in pronounciation if not i.isdigit()]).lower()
            word_pronounciations.append(pronounciation.split(" "))
    
    for index, pronounciation in enumerate(word_pronounciations):
        for phoneme_index, phoneme in enumerate(pronounciation):
            word_pronounciations[index][phoneme_index] = random_phoneme(phoneme, prob_viseme, prob_different)
        
    word_pronounciations = ["-".join(phoneme for phoneme in pronounciation) for pronounciation in word_pronounciations]
    return " ".join(word_pronounciations)

In [154]:
string_to_phonemes(splits[0], 0.3, 0)

'g-uh-d iy-f-n-ih-ng t-ay-d-ah-t ay uw-ao-n-l l-uw t-ao-hh ah-b-aw-l dh-ah d-ay-b-ey-d b-ih-n hh-ae-f-iy-ng ih-t w-aa-sh-ih-ng-t-ay-n ow-v-er dh-ah n-ae-sh-ah-n-ah-l d-ey-t ah l-ay-b-ey-t th-ae-t d-er-eh-hh-l-t-iy ah-f-eh-hh-t-s dh-ah l-ih-v-z ah-f ao-l ay-p-ae-r-ah-hh-ah-t-s'

In [164]:
iteration_count = 5

scores = []
for i in range(split_count):
    l1 = []
    for j in range(iteration_count):
        l1.append({})
    scores.append(l1)

scores

[[{}, {}, {}, {}, {}],
 [{}, {}, {}, {}, {}],
 [{}, {}, {}, {}, {}],
 [{}, {}, {}, {}, {}],
 [{}, {}, {}, {}, {}],
 [{}, {}, {}, {}, {}]]

In [156]:
import openai
import os
import dotenv

dotenv.load_dotenv(".env")

client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [165]:
import numpy as np
from tqdm import tqdm
import jiwer
import rapidfuzz

transforms = jiwer.Compose(
    [
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
)

def evaluate_model_quality(text, confusion_rate):
    phonemes = string_to_phonemes(text, confusion_rate, 0.02)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": f"""You are given a list of phonemes of a person talking. To the best of your ability, transcribe them into English. 
                Choose the likeliest speech. Put your final transcription within <transcription> </transcription> tags. 
                Remember that, in your input, these could be confused - so don't adhere to the phonemes you are given 100%.
                {expl}"""
            },
            {
                "role": "user",
                "content": phonemes
            }
        ]
    )
    response = response.choices[0].message.content
    transcription = response.split("<transcription>")[1].split("</transcription>")[0]
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": f"""
                You were given a list of phonemes of a person talking. To the best of your ability, transcribe them into English. 
                Choose the likeliest speech.
                Remember that, in your input, these could be confused - so don't adhere to the phonemes you are given 100%.
                {expl}

                """
            },
            {
                "role": "user",
                "content": phonemes
            },
            {
                "role": "assistant",
                "content": transcription
            },
            {
                "role": "user",
                "content": "Now, please revise your earlier transcription. Put your final transcription within <transcription> </transcription> tags. "
            }
        ]
    )
    response = response.choices[0].message.content

    transcription = response.split("<transcription>")[1].split("</transcription>")[0]
    score = jiwer.wer(text, transcription, truth_transform=transforms, hypothesis_transform=transforms)
    return score

In [166]:
evaluate_model_quality(splits[0], 0.2)

0.27586206896551724

In [167]:
for iteration in tqdm(range(iteration_count)):
    for phoneme_viseme_confusion_rate in [0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.45, 0.6]:
        for split_index, split in tqdm(enumerate(splits)):
            if not(phoneme_viseme_confusion_rate in scores[split_index][iteration]):
                scores[split_index][iteration][phoneme_viseme_confusion_rate] = evaluate_model_quality(split, phoneme_viseme_confusion_rate)

6it [00:16,  2.73s/it]00:00<?, ?it/s]
6it [00:17,  2.87s/it]
6it [00:16,  2.71s/it]
6it [00:17,  2.96s/it]
6it [00:21,  3.58s/it]
6it [00:16,  2.77s/it]
6it [00:19,  3.17s/it]
6it [00:19,  3.22s/it]
6it [00:21,  3.65s/it]02:24<09:36, 144.21s/it]
6it [00:16,  2.75s/it]
6it [00:17,  2.98s/it]
6it [00:16,  2.75s/it]
6it [00:16,  2.75s/it]
6it [00:18,  3.02s/it]
6it [00:17,  2.86s/it]
6it [00:17,  2.84s/it]
6it [00:15,  2.62s/it]04:45<07:08, 142.69s/it]
6it [00:16,  2.75s/it]
6it [00:21,  3.56s/it]
6it [00:15,  2.67s/it]
6it [00:16,  2.74s/it]
6it [00:18,  3.05s/it]
6it [00:18,  3.16s/it]
6it [00:20,  3.49s/it]
6it [00:17,  2.98s/it]07:10<04:46, 143.39s/it]
6it [00:15,  2.65s/it]
6it [00:20,  3.43s/it]
6it [00:19,  3.23s/it]
6it [00:19,  3.30s/it]
6it [00:18,  3.03s/it]
6it [00:24,  4.14s/it]
6it [00:25,  4.28s/it]
6it [00:22,  3.71s/it]09:52<02:30, 150.85s/it]
6it [00:18,  3.15s/it]
6it [00:17,  2.85s/it]
6it [00:17,  2.87s/it]
6it [00:14,  2.39s/it]
6it [00:18,  3.03s/it]
6it [00:16,  2.

In [168]:
print(completions[0][0][0.0])

KeyError: 0.0

In [169]:
scores

[[{0.0: 0.06896551724137931,
   0.05: 0.10344827586206896,
   0.1: 0.13793103448275862,
   0.15: 0.27586206896551724,
   0.2: 0.3103448275862069,
   0.3: 0.4482758620689655,
   0.45: 0.7586206896551724,
   0.6: 0.7241379310344828},
  {0.0: 0.06896551724137931,
   0.05: 0.10344827586206896,
   0.1: 0.1724137931034483,
   0.15: 0.13793103448275862,
   0.2: 0.27586206896551724,
   0.3: 0.3448275862068966,
   0.45: 0.6206896551724138,
   0.6: 0.7931034482758621},
  {0.0: 0.10344827586206896,
   0.05: 0.10344827586206896,
   0.1: 0.10344827586206896,
   0.15: 0.1724137931034483,
   0.2: 0.13793103448275862,
   0.3: 0.27586206896551724,
   0.45: 0.41379310344827586,
   0.6: 0.6896551724137931},
  {0.0: 0.10344827586206896,
   0.05: 0.06896551724137931,
   0.1: 0.10344827586206896,
   0.15: 0.10344827586206896,
   0.2: 0.20689655172413793,
   0.3: 0.27586206896551724,
   0.45: 0.6896551724137931,
   0.6: 0.6896551724137931},
  {0.0: 0.10344827586206896,
   0.05: 0.27586206896551724,
   0.1: 0

In [171]:
averages = {}
for a in scores:
    for b in a:
        for k in b:
            if not(k in averages):
                averages[k] = 0
            averages[k] += b[k] / (len(a) * (len(b)))

print(averages)

{0.0: 0.0562193326494723, 0.05: 0.0733103426586618, 0.1: 0.09379213658059633, 0.15: 0.1308108874300214, 0.2: 0.2000139596616519, 0.3: 0.29565760043605344, 0.45: 0.48407505029541603, 0.6: 0.5993022951895653}
